In [1]:
import numpy as np
import pandas as pd

In [2]:
def remove_digit_and_parenthesis(x):
    
    res = ''.join([i for i in x if not i.isdigit()])
    if res.__contains__('('):
        n = res.index('(')
        res = res[:n-1]
    return res
def answer_one():
    
    file_name= 'Energy Indicators.xls'
    energy = pd.read_excel(file_name,usecols=[2,3,4,5],index_col=None)
    energy = energy[18:244].reset_index(drop = True)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    energy['Energy Supply'] = [x * 1000000 if x != '...'else x for x in energy['Energy Supply'] ] 
    energy.replace('...',np.nan, inplace = True)

    energy.replace(['Republic of Korea','United States of America20','United Kingdom of Great Britain and Northern Ireland19','China, Hong Kong Special Administrative Region3'],
                   ['South Korea','United States','United Kingdom','Hong Kong'],
                   inplace = True)
    energy.replace(['Bolivia (Plurinational State of)', 'Switzerland17'],
                   ['Bolivia','Switzerland'],
                   inplace = True)
    energy['Country'] = energy['Country'].apply(remove_digit_and_parenthesis)

    file_name=  'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4701247.csv'
    GDP = pd.read_csv(file_name, sep = ',', header= [0],
                     dtype = None, skiprows=4, encoding='utf-8')
    GDP.replace(['Korea, Rep.','Iran, Islamic Rep.','Hong Kong SAR, China'],
                   ['South Korea','Iran','Hong Kong'],
                   inplace = True)
    GDP = pd.merge(GDP['Country Name'], GDP.loc[:, '2006':'2015'],left_index=True, right_index=True)
    file_name= 'scimagojr country rank 1996-2021.xlsx'
    ScimEn = pd.read_excel(file_name,index_col=None)
    ScimEn.drop('Region',axis=1,inplace = True)
    ScimEn = ScimEn.head(15)
    energy_GDP = pd.merge(energy, GDP, how ='inner',left_on = 'Country', right_on= 'Country Name')

    df = pd.merge(ScimEn,energy_GDP, how ='left',left_on = 'Country', right_on= 'Country').set_index('Country')
    df = df.drop('Country Name',axis=1)
    return df

answer_one()


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.02,273,1.271910e+11,93.0,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,184851,181106,2623922,819242,14.19,389,9.083800e+10,286.0,11.570980,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,60257,58589,590570,213760,9.80,198,3.319500e+10,26.0,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,52780,52281,557023,132113,10.55,203,1.898400e+10,149.0,10.232820,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,47141,45928,748994,132737,15.89,244,7.920000e+09,124.0,10.600470,2.717060e+12,3.106182e+12,2.938882e+12,2.425798e+12,2.491110e+12,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12
Germany,6,42343,41464,528645,115432,12.48,211,1.326100e+10,165.0,17.901530,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,39424,39189,142937,67935,3.63,96,3.070900e+10,214.0,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,35588,34940,665415,113363,18.70,246,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,31260,29959,433388,105641,13.86,177,6.530000e+09,109.0,33.667230,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


In [3]:
def answer_two():
    Top15 = answer_one()
    avgGDP = Top15.loc[:, '2006':'2015'].mean(1).sort_values(ascending=False)
    return avgGDP
answer_two()

Country
United States         1.570403e+13
China                 6.927702e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.792011e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400727e+12
South Korea           1.221372e+12
Australia             1.207106e+12
Iran                  4.563261e+11
dtype: float64

In [4]:
def answer_three():
    Top15 = answer_one()
    six_country = answer_two().index[5]
    return Top15.loc[six_country]['2015'] - Top15.loc[six_country]['2006']
answer_three()

118652421857.7998

In [5]:
def answer_four():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Citations']/Top15['Self-citations']
    name = Top15.sort_values(by='Ratio',ascending=False).index[0]
    ratio_largest = Top15.sort_values(by='Ratio',ascending=False).iloc[0]['Ratio']
    return (name,ratio_largest)
answer_four()

('Australia', 6.319360704373624)

In [6]:
def answer_five():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    return Top15.sort_values(by='Population',ascending=False).index[0]
answer_five()

'China'

In [7]:
def answer_six():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Citable documents per person'] = Top15['Citable documents']/Top15['Population']
    return Top15['Citable documents per person'].corr(Top15['Energy Supply per Capita'])
answer_six()

0.7272331912434825

In [8]:
def answer_seven():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Continents = ['Asia', 'Australia', 'Europe', 'North America', 'South America']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df = pd.DataFrame()
    df.index = Continents
    sizes = []
    sums=[]
    means=[]
    std=[]
    for x in Continents:
        df_current_continent = Top15.loc[[c for c, d in ContinentDict.items() if d == x]]
        sizes.append(len(df_current_continent))
        sums.append(df_current_continent['Population'].sum())
        means.append(df_current_continent['Population'].mean())
        std.append(df_current_continent['Population'].std())
    df['Size'] = sizes
    df['Sum'] = sums
    df['Mean'] = means
    df['Std deviation'] = std
    return df
answer_seven()

,Size,Sum,Mean,Std deviation
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
